In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import models
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score
from skimage import transform
from torch.utils import data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch
from tqdm import tqdm
from colorize.util import reshape

In [ ]:
def comparison_model_metrics(model, model_name, val_loader, val_name, index_mapper=None):
    """Get the model metrics (non-balanced AUC, balanced AUC, accuracy) for a comparison model on validation dataset"""
    Y_pred = []
    y_true = []
    
    for X, y in tqdm(val_loader):
        output = model(X)
        if index_mapper is not None:
            output = output[index_mapper]
        Y_pred.append(output.softmax(dim=-1).detach().numpy())
        y_true.append(y.numpy())
        
    Y_pred = np.vstack(Y_pred)
    y_true = np.concatenate(y_true)
    y_pred_labels = np.argmax(Y_pred, axis=1)
    
    np.save(f'data/{model_name}_{val_name}_Y_pred', Y_pred)
    np.save(f'data/{model_name}_{val_name}_y_true', y_true)
    np.save(f'data/{model_name}_{val_name}_y_pred_labels', y_pred_labels)

    acc = accuracy_score(y_true, y_pred_labels)
    
    return acc

In [ ]:
# Load Inception V3 model pretrained on Imagenet 
inceptionV3 = models.inception_v3(pretrained=True)

In [ ]:
# Load VGG16 Batch Norm model pretrained on Imagenet
VGG16 = models.vgg16_bn(pretrained=True)

In [ ]:
# Folder for Imagenet validation set 
imagenet_val_folder = 'data/ILSVRC2012_img_val/'

# Dataloader constants 
BS = 40
N_WORKERS = 1

# Normalizer for Imagenet
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(imagenet_val_folder, transforms.Compose([
            transforms.Scale(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=BS, shuffle=False,
        num_workers=N_WORKERS, pin_memory=True)

In [ ]:
# Folder for dogs dataset 
dogs_val_folder = 'data/test/'

# Dataloader constants 
BS = 2
N_WORKERS = 1

# Normalizer for Imagenet
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

dogs_val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(dogs_val_folder, transforms.Compose([
            transforms.Scale(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=BS, shuffle=False,
        num_workers=N_WORKERS, pin_memory=True)

# Class to index mapping for dogs dataloader
dogs_class_to_idx = dogs_val_loader.dataset.class_to_idx
dogs_idx_to_class = {v: k.split('-')[0] for k, v in dogs_class_to_idx.items()}

# Class to index mapping for imagenet dataloader
imagenet_class_to_idx = imagenet_val_loader.dataset.class_to_idx

# Mapper array to slice imagenet output and align with dogs dataset label 
imagenet_to_dogs = np.zeros(len(dogs_class_to_idx), dtype=int)
for i in range(len(imagenet_to_dogs)):
    imagenet_to_dogs[i] = imagenet_class_to_idx[dogs_idx_to_class[i]]

In [ ]:
model_metrics_imagenet = comparison_model_metrics(VGG16, 'VGG16BN', val_loader_imagenet, 'imagenet')

In [ ]:
model_metrics_dogs = comparison_model_metrics(VGG16, 'VGG16BN', val_loader_dogs, 'dogs', imagenet_to_dogs)